In [ ]:
from bs4 import BeautifulSoup
import requests
from urllib.request import Request, urlopen
import pandas as pd
import requests

from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver

import time

import re
import unicodedata

In [ ]:
imdb = pd.read_csv('../data/movie_lense/links.csv')

In [ ]:
## Main_url
## Get top summary and bttom summary from main page

def get_main(main_url):

    page = requests.get(main_url)
    soup = BeautifulSoup(page.text, 'html')
    top_summary = soup.find_all('div', attrs = {'class': 'plot_summary'})[0].find('div', attrs = {'class': 'summary_text'}).text.strip()
    
    ## Bottom summary is also on the summaries page
    bot_summary = soup.find_all('div', attrs = {'id': 'titleStoryLine'})[0].find_all('div', attrs = {'class': 'inline canwrap'})[0].find('span').text.strip()
    return (top_summary, bot_summary)

In [ ]:
## Summary_url
## Get list of summaries and synopsis
## Missing for a lot of movies
## One of these summaries is same as bottom_summary


def get_summaries(summmary_url):
    
    page = requests.get(summary_url)
    soup = BeautifulSoup(page.text, 'html')

    summaries = []
    table = soup.find_all('ul', attrs = {'id': 'plot-summaries-content'})[0].find_all('p')
    for i in table:
        summaries.append(i.text.strip())
    
    ## Synopsis not there
    blank_synopsis = 'It looks like we don\'t have a Synopsis for this title yet. Be the first to contribute! Just click the "Edit page" button at the bottom of the page or learn more in the Synopsis submission guide.'

    synopsis = soup.find_all('ul', attrs = {'id': 'plot-synopsis-content'})[0].find('li').text.strip()
    if synopsis == blank_synopsis:
        synopsis = ''
    return (summaries, synopsis)

In [ ]:
imdb['top_summary'] = ''
imdb['bot_summary'] = ''
imdb['summaries'] = ''
imdb['synopsis'] = ''

## Loop through each movie and append to dataframe with summaries and synopsis
## Need to still add reviews

for i in range(len(imdb)):
    main_url = 'https://www.imdb.com/title/tt00' + str(imdb['imdbId'][i]) + '/'
    summary_url = 'https://www.imdb.com/title/tt00' + str(imdb['imdbId'][i]) + '/plotsummary'
    review_url = 'https://www.imdb.com/title/tt00' + str(imdb['imdbId'][i]) + '/reviews?ref_=tt_urv'
    
    try:
        top, bot = get_main(main_url)
        imdb.loc[i, 'top_summary'] = top
        imdb.loc[i, 'bot_summary'] = bot

        summaries, synopsis = get_summaries(summary_url)
        imdb.at[i, 'summaries'] = summaries
        imdb.loc[i, 'synopsis'] = synopsis
    
    except:
        print("failed: ", i)
    
    if (i%100) == 0:
        print(i)

In [7]:
imdb.to_csv('../data/movie_lense/summaries.csv')